# ChromaとOpenAIを使った堅牢な質問応答システム

このノートブックでは、オープンソースの埋め込みデータベースである[Chroma](https://trychroma.com)と、OpenAIの[テキスト埋め込み](https://platform.openai.com/docs/guides/embeddings/use-cases)および[チャット補完](https://platform.openai.com/docs/guides/chat) APIを使用して、データコレクションに関する質問に答える方法をステップバイステップで説明します。

さらに、このノートブックでは質問応答システムをより堅牢にするためのトレードオフについても説明します。これから見るように、*単純なクエリが常に最良の結果を生み出すとは限りません*！

## LLMを使った質問応答

OpenAIのChatGPTのような大規模言語モデル（LLM）は、モデルが訓練されていない、またはアクセスできないデータに関する質問に答えるために使用できます。例えば：

- メールやメモなどの個人データ
- アーカイブや法的文書などの高度に専門化されたデータ
- 最近のニュース記事などの新しく作成されたデータ

この制限を克服するために、LLM自体と同様に自然言語でのクエリに適したデータストアを使用できます。Chromaのような埋め込みストアは、文書自体と並んで文書を[埋め込み](https://openai.com/blog/introducing-text-and-code-embeddings)として表現します。

テキストクエリを埋め込むことで、Chromaは関連する文書を見つけることができ、それをLLMに渡して質問に答えることができます。このアプローチの詳細な例とバリエーションを紹介します。

# セットアップと準備

まず、必要なPython依存関係がインストールされていることを確認します。

In [1]:
%pip install -qU openai chromadb pandas

Note: you may need to restart the kernel to use updated packages.


このノートブック全体でOpenAIのAPIを使用します。APIキーは[https://beta.openai.com/account/api-keys](https://beta.openai.com/account/api-keys)から取得できます。

ターミナルで`export OPENAI_API_KEY=sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx`コマンドを実行することで、APIキーを環境変数として追加できます。環境変数がまだ設定されていない場合は、ノートブックを再読み込みする必要があることに注意してください。または、以下に示すようにノートブック内で設定することもできます。

In [2]:
import os
from openai import OpenAI

# Uncomment the following line to set the environment variable in the notebook
# os.environ["OPENAI_API_KEY"] = 'sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    client = OpenAI(api_key=api_key)
    print("OpenAI client is ready")
else:
    print("OPENAI_API_KEY environment variable not found")

OpenAI client is ready


In [3]:
# Set the model for all API calls
OPENAI_MODEL = "gpt-4o"

# データセット

このノートブック全体を通して、[SciFact dataset](https://github.com/allenai/scifact)を使用します。これは専門家によって注釈付けされた科学的主張のキュレートされたデータセットで、論文のタイトルと要約のテキストコーパスが付属しています。各主張は、コーパス内の文書に従って、支持される、矛盾する、またはいずれの方向にも十分な証拠がない、のいずれかに分類されます。

コーパスが正解として利用可能であることにより、LLMの質問応答に対する以下のアプローチがどの程度うまく機能するかを調査することができます。

In [4]:
# Load the claim dataset
import pandas as pd

data_path = '../../data'

claim_df = pd.read_json(f'{data_path}/scifact_claims.jsonl', lines=True)
claim_df.head()

,id,claim,evidence,cited_doc_ids
0,1,0-dimensional biomaterials show inductive prop...,{},[31715818]
1,3,"1,000 genomes project enables mapping of genet...","{'14717500': [{'sentences': [2, 5], 'label': '...",[14717500]
2,5,1/2000 in UK have abnormal PrP positivity.,"{'13734012': [{'sentences': [4], 'label': 'SUP...",[13734012]
3,13,5% of perinatal mortality is due to low birth ...,{},[1606628]
4,36,A deficiency of vitamin B12 increases blood le...,{},"[5152028, 11705328]"


# モデルに直接質問する

ChatGPTは大量の科学的情報で訓練されています。ベースラインとして、追加のコンテキストなしでモデルがすでに何を知っているかを理解したいと思います。これにより、全体的なパフォーマンスを較正することができます。

いくつかの事実例を含む適切なプロンプトを構築し、データセット内の各主張についてモデルに問い合わせます。モデルには、主張を「True」、「False」、または証拠が不十分な場合は「NEE」として評価するよう求めます。

In [5]:
def build_prompt(claim):
    return [
        {"role": "system", "content": "I will ask you to assess a scientific claim. Output only the text 'True' if the claim is true, 'False' if the claim is false, or 'NEE' if there's not enough evidence."},
        {"role": "user", "content": f"""
Example:

Claim:
0-dimensional biomaterials show inductive properties.

Assessment:
False

Claim:
1/2000 in UK have abnormal PrP positivity.

Assessment:
True

Claim:
Aspirin inhibits the production of PGE2.

Assessment:
False

End of examples. Assess the following claim:

Claim:
{claim}

Assessment:
"""}
    ]


def assess_claims(claims):
    responses = []
    # Query the OpenAI API
    for claim in claims:
        response = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=build_prompt(claim),
            max_tokens=3,
        )
        # Strip any punctuation or whitespace from the response
        responses.append(response.choices[0].message.content.strip('., '))

    return responses

データセットから50件のクレームをサンプリングします

In [6]:
# Let's take a look at 50 claims
samples = claim_df.sample(50)

claims = samples['claim'].tolist()


データセットに従って正解データを評価します。データセットの説明によると、各主張は証拠によって支持されるか反駁されるか、あるいはどちらの方向にも十分な証拠が存在しないかのいずれかです。

In [7]:
def get_groundtruth(evidence):
    groundtruth = []
    for e in evidence:
        # Evidence is empty
        if len(e) == 0:
            groundtruth.append('NEE')
        else:
            # In this dataset, all evidence for a given claim is consistent, either SUPPORT or CONTRADICT
            if list(e.values())[0][0]['label'] == 'SUPPORT':
                groundtruth.append('True')
            else:
                groundtruth.append('False')
    return groundtruth

In [8]:
evidence = samples['evidence'].tolist()
groundtruth = get_groundtruth(evidence)

また、モデルの評価と正解データを比較した混同行列を、読みやすい表形式で出力します。

In [9]:
def confusion_matrix(inferred, groundtruth):
    assert len(inferred) == len(groundtruth)
    confusion = {
        'True': {'True': 0, 'False': 0, 'NEE': 0},
        'False': {'True': 0, 'False': 0, 'NEE': 0},
        'NEE': {'True': 0, 'False': 0, 'NEE': 0},
    }
    for i, g in zip(inferred, groundtruth):
        confusion[i][g] += 1

    # Pretty print the confusion matrix
    print('\tGroundtruth')
    print('\tTrue\tFalse\tNEE')
    for i in confusion:
        print(i, end='\t')
        for g in confusion[i]:
            print(confusion[i][g], end='\t')
        print()

    return confusion

モデルに対して、追加のコンテキストなしに、主張を直接評価するよう求めます。

In [10]:
gpt_inferred = assess_claims(claims)
confusion_matrix(gpt_inferred, groundtruth)

	Groundtruth
	True	False	NEE
True	9	3	15	
False	0	3	2	
NEE	8	6	4	


{'True': {'True': 9, 'False': 3, 'NEE': 15},
 'False': {'True': 0, 'False': 3, 'NEE': 2},
 'NEE': {'True': 8, 'False': 6, 'NEE': 4}}

## 結果

これらの結果から、LLMは主張を真実として評価する強いバイアスを持っており、それらが偽である場合でも真実と評価し、また偽の主張を証拠不十分として評価する傾向があることがわかります。なお、「証拠不十分」とは、追加の文脈なしに、真空状態での主張に対するモデルの評価に関するものです。

# コンテキストの追加

ここで、論文タイトルと要約のコーパスから利用可能な追加のコンテキストを追加します。このセクションでは、OpenAIのテキスト埋め込みを使用して、テキストコーパスをChromaに読み込む方法を示します。

まず、テキストコーパスを読み込みます。

In [11]:
# Load the corpus into a dataframe
corpus_df = pd.read_json(f'{data_path}/scifact_corpus.jsonl', lines=True)
corpus_df.head()

,doc_id,title,abstract,structured
0,4983,Microstructural development of human newborn c...,[Alterations of the architecture of cerebral w...,False
1,5836,Induction of myelodysplasia by myeloid-derived...,[Myelodysplastic syndromes (MDS) are age-depen...,False
2,7912,"BC1 RNA, the transcript from a master gene for...",[ID elements are short interspersed elements (...,False
3,18670,The DNA Methylome of Human Peripheral Blood Mo...,[DNA methylation plays an important role in bi...,False
4,19238,The human myelin basic protein gene is include...,[Two human Golli (for gene expressed in the ol...,False


## コーパスをChromaに読み込む

次のステップは、コーパスをChromaに読み込むことです。埋め込み関数が与えられると、Chromaは各ドキュメントの埋め込みを自動的に処理し、そのテキストとメタデータと一緒に保存するため、クエリを簡単に実行できます。

私たちは（一時的な）Chromaクライアントをインスタンス化し、SciFact のタイトルと要約コーパス用のコレクションを作成します。
Chromaは永続化設定でもインスタンス化できます。詳細は[Chroma docs](https://docs.trychroma.com/usage-guide?lang=py)で学習してください。

In [12]:
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

# We initialize an embedding function, and provide it to the collection.
embedding_function = OpenAIEmbeddingFunction(api_key=os.getenv("OPENAI_API_KEY"))

chroma_client = chromadb.Client() # Ephemeral by default
scifact_corpus_collection = chroma_client.create_collection(name='scifact_corpus', embedding_function=embedding_function)

次に、コーパスをChromaに読み込みます。このデータ読み込みはメモリ集約的であるため、50-1000件のバッチに分けたバッチ読み込み方式を使用することをお勧めします。この例では、コーパス全体の処理に1分強かかるはずです。先ほど指定した`embedding_function`を使用して、バックグラウンドで自動的に埋め込み処理が行われています。

In [13]:
batch_size = 100

for i in range(0, len(corpus_df), batch_size):
    batch_df = corpus_df[i:i+batch_size]
    scifact_corpus_collection.add(
        ids=batch_df['doc_id'].apply(lambda x: str(x)).tolist(), # Chroma takes string IDs.
        documents=(batch_df['title'] + '. ' + batch_df['abstract'].apply(lambda x: ' '.join(x))).to_list(), # We concatenate the title and abstract.
        metadatas=[{"structured": structured} for structured in batch_df['structured'].to_list()] # We also store the metadata, though we don't use it in this example.
    )

## コンテキストの取得

次に、サンプル内の各クレームに関連する可能性のある文書をコーパスから取得します。これらをLLMがクレームを評価する際のコンテキストとして提供したいと考えています。埋め込み距離に基づいて、各クレームに対して最も関連性の高い3つの文書を取得します。

In [14]:
claim_query_result = scifact_corpus_collection.query(query_texts=claims, include=['documents', 'distances'], n_results=3)

コーパスから取得した追加のコンテキストを考慮して、新しいプロンプトを作成します。

In [15]:
def build_prompt_with_context(claim, context):
    return [{'role': 'system', 'content': "I will ask you to assess whether a particular scientific claim, based on evidence provided. Output only the text 'True' if the claim is true, 'False' if the claim is false, or 'NEE' if there's not enough evidence."},
            {'role': 'user', 'content': f""""
The evidence is the following:

{' '.join(context)}

Assess the following claim on the basis of the evidence. Output only the text 'True' if the claim is true, 'False' if the claim is false, or 'NEE' if there's not enough evidence. Do not output any other text.

Claim:
{claim}

Assessment:
"""}]


def assess_claims_with_context(claims, contexts):
    responses = []
    # Query the OpenAI API
    for claim, context in zip(claims, contexts):
        # If no evidence is provided, return NEE
        if len(context) == 0:
            responses.append('NEE')
            continue
        response = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=build_prompt_with_context(claim=claim, context=context),
            max_tokens=3,
        )
        # Strip any punctuation or whitespace from the response
        responses.append(response.choices[0].message.content.strip('., '))

    return responses

次に、取得したコンテキストを使用して主張を評価するようにモデルに依頼します。

In [16]:
gpt_with_context_evaluation = assess_claims_with_context(claims, claim_query_result['documents'])
confusion_matrix(gpt_with_context_evaluation, groundtruth)

	Groundtruth
	True	False	NEE
True	13	1	4	
False	1	10	2	
NEE	3	1	15	


{'True': {'True': 13, 'False': 1, 'NEE': 4},
 'False': {'True': 1, 'False': 10, 'NEE': 2},
 'NEE': {'True': 3, 'False': 1, 'NEE': 15}}

## 結果

モデル全体的により良いパフォーマンスを示し、偽の主張を正しく識別することが大幅に改善されました。さらに、ほとんどのNEEケースも正しく識別されるようになりました。

取得された文書を見ると、時々主張に関連していないことがあります。これにより、モデルは余分な情報に混乱し、情報が無関係であっても十分な証拠が存在すると判断してしまう可能性があります。これは、常に「最も」関連性の高い3つの文書を要求するためですが、これらの文書は特定の時点を超えると全く関連性がない可能性があります。

## 関連性に基づくコンテキストのフィルタリング

Chromaは文書自体とともに距離スコアも返します。距離に対して閾値を設定することで、モデルに提供するコンテキストに含まれる無関係な文書を減らすことができます。

閾値でフィルタリングした後にコンテキスト文書が残らない場合は、モデルをバイパスして、十分な証拠がないことを単純に返します。

In [17]:
def filter_query_result(query_result, distance_threshold=0.25):
# For each query result, retain only the documents whose distance is below the threshold
    for ids, docs, distances in zip(query_result['ids'], query_result['documents'], query_result['distances']):
        for i in range(len(ids)-1, -1, -1):
            if distances[i] > distance_threshold:
                ids.pop(i)
                docs.pop(i)
                distances.pop(i)
    return query_result


In [18]:
filtered_claim_query_result = filter_query_result(claim_query_result)

今度は、このより明確なコンテキストを使用して主張を評価します。

In [19]:
gpt_with_filtered_context_evaluation = assess_claims_with_context(claims, filtered_claim_query_result['documents'])
confusion_matrix(gpt_with_filtered_context_evaluation, groundtruth)

	Groundtruth
	True	False	NEE
True	9	0	1	
False	0	7	0	
NEE	8	5	20	


{'True': {'True': 9, 'False': 0, 'NEE': 1},
 'False': {'True': 0, 'False': 7, 'NEE': 0},
 'NEE': {'True': 8, 'False': 5, 'NEE': 20}}

## 結果

モデルは現在、十分な証拠が存在しない場合に、TrueまたはFalseと評価する主張の数を大幅に減らしています。しかし、モデルは非常に慎重になり、ほとんどの項目を証拠不十分とラベル付けする傾向があり、確実性から遠ざかる方向にバイアスがかかっています。現在、ほとんどの主張が証拠不十分と評価されています。これは、それらの大部分が距離閾値によってフィルタリングされているためです。最適な動作点を見つけるために距離閾値を調整することは可能ですが、これは困難であり、データセットと埋め込みモデルに依存します。

# 仮想文書埋め込み：幻覚を生産的に活用する

私たちは、モデルを混乱させる可能性のある関連性の低い文書を取得することなく、関連する文書を取得できるようになりたいと考えています。これを実現する一つの方法は、検索クエリを改善することです。

これまで、私たちは科学論文を説明する_要約_を含むコーパスに対して、単一の文による陳述である_主張_を使用してデータセットにクエリを実行してきました。直感的には、これらは関連している可能性がありますが、その構造と意味には大きな違いがあります。これらの違いは埋め込みモデルによってエンコードされ、クエリと最も関連性の高い結果との間の距離に影響を与えます。

私たちは、LLMの力を活用して関連するテキストを生成することで、この問題を克服できます。事実は幻覚かもしれませんが、モデルが生成する文書の内容と構造は、クエリよりも私たちのコーパス内の文書により類似しています。これにより、より良いクエリが得られ、ひいてはより良い結果が得られる可能性があります。

このアプローチは[仮想文書埋め込み（HyDE）](https://arxiv.org/abs/2212.10496)と呼ばれ、検索タスクにおいて非常に優れていることが示されています。これにより、コンテキストを汚染することなく、より関連性の高い情報をコンテキストに取り込むことができるはずです。

要約：
- 単一の文ではなく完全な要約を埋め込む場合、はるかに良いマッチングが得られる
- しかし、主張は通常単一の文である
- そこでHyDEは、GPT3を使用して主張を幻覚的な要約に拡張し、それらの要約に基づいて検索することが（主張 -> 要約 -> 結果）、直接検索する（主張 -> 結果）よりも効果的であることを示している

まず、評価したい各主張について、コーパス内の文書と類似した文書を生成するよう、文脈内の例を使用してモデルにプロンプトを与えます。

In [20]:
def build_hallucination_prompt(claim):
    return [{'role': 'system', 'content': """I will ask you to write an abstract for a scientific paper which supports or refutes a given claim. It should be written in scientific language, include a title. Output only one abstract, then stop.

    An Example:

    Claim:
    A high microerythrocyte count raises vulnerability to severe anemia in homozygous alpha (+)- thalassemia trait subjects.

    Abstract:
    BACKGROUND The heritable haemoglobinopathy alpha(+)-thalassaemia is caused by the reduced synthesis of alpha-globin chains that form part of normal adult haemoglobin (Hb). Individuals homozygous for alpha(+)-thalassaemia have microcytosis and an increased erythrocyte count. Alpha(+)-thalassaemia homozygosity confers considerable protection against severe malaria, including severe malarial anaemia (SMA) (Hb concentration < 50 g/l), but does not influence parasite count. We tested the hypothesis that the erythrocyte indices associated with alpha(+)-thalassaemia homozygosity provide a haematological benefit during acute malaria.
    METHODS AND FINDINGS Data from children living on the north coast of Papua New Guinea who had participated in a case-control study of the protection afforded by alpha(+)-thalassaemia against severe malaria were reanalysed to assess the genotype-specific reduction in erythrocyte count and Hb levels associated with acute malarial disease. We observed a reduction in median erythrocyte count of approximately 1.5 x 10(12)/l in all children with acute falciparum malaria relative to values in community children (p < 0.001). We developed a simple mathematical model of the linear relationship between Hb concentration and erythrocyte count. This model predicted that children homozygous for alpha(+)-thalassaemia lose less Hb than children of normal genotype for a reduction in erythrocyte count of >1.1 x 10(12)/l as a result of the reduced mean cell Hb in homozygous alpha(+)-thalassaemia. In addition, children homozygous for alpha(+)-thalassaemia require a 10% greater reduction in erythrocyte count than children of normal genotype (p = 0.02) for Hb concentration to fall to 50 g/l, the cutoff for SMA. We estimated that the haematological profile in children homozygous for alpha(+)-thalassaemia reduces the risk of SMA during acute malaria compared to children of normal genotype (relative risk 0.52; 95% confidence interval [CI] 0.24-1.12, p = 0.09).
    CONCLUSIONS The increased erythrocyte count and microcytosis in children homozygous for alpha(+)-thalassaemia may contribute substantially to their protection against SMA. A lower concentration of Hb per erythrocyte and a larger population of erythrocytes may be a biologically advantageous strategy against the significant reduction in erythrocyte count that occurs during acute infection with the malaria parasite Plasmodium falciparum. This haematological profile may reduce the risk of anaemia by other Plasmodium species, as well as other causes of anaemia. Other host polymorphisms that induce an increased erythrocyte count and microcytosis may confer a similar advantage.

    End of example.

    """}, {'role': 'user', 'content': f""""
    Perform the task for the following claim.

    Claim:
    {claim}

    Abstract:
    """}]


def hallucinate_evidence(claims):
    responses = []
    # Query the OpenAI API
    for claim in claims:
        response = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=build_hallucination_prompt(claim),
        )
        responses.append(response.choices[0].message.content)
    return responses

各クレームに対してドキュメントを幻覚生成します。

*注意：これには時間がかかる場合があります。100件のクレームで約7分程度*。より迅速に結果を得るために、評価したいクレーム数を減らすことができます。

In [21]:
hallucinated_evidence = hallucinate_evidence(claims)

幻覚された文書をコーパスへのクエリとして使用し、同じ距離閾値を使って結果をフィルタリングします。

In [22]:
hallucinated_query_result = scifact_corpus_collection.query(query_texts=hallucinated_evidence, include=['documents', 'distances'], n_results=3)
filtered_hallucinated_query_result = filter_query_result(hallucinated_query_result)

その後、新しいコンテキストを使用して、モデルに主張を評価するよう求めます。

In [23]:
gpt_with_hallucinated_context_evaluation = assess_claims_with_context(claims, filtered_hallucinated_query_result['documents'])
confusion_matrix(gpt_with_hallucinated_context_evaluation, groundtruth)

	Groundtruth
	True	False	NEE
True	13	0	3	
False	1	10	1	
NEE	3	2	17	


{'True': {'True': 13, 'False': 0, 'NEE': 3},
 'False': {'True': 1, 'False': 10, 'NEE': 1},
 'NEE': {'True': 3, 'False': 2, 'NEE': 17}}

## 結果

HyDEをシンプルな距離閾値と組み合わせることで、大幅な改善が得られます。モデルはもはや主張をTrueと評価することに偏ることもなく、証拠が不十分であると判断することに偏ることもありません。また、証拠が不十分な場合をより正確に評価できるようになります。

# 結論

文書のコーパスに基づくコンテキストをLLMに装備することは、LLMの一般的な推論と自然言語インタラクションを独自のデータに適用するための強力な技術です。しかし、単純なクエリと検索では最良の結果が得られない可能性があることを理解しておくことが重要です！最終的に、データを理解することが、検索ベースの質問応答アプローチから最大限の効果を得るのに役立ちます。